In [2]:
%matplotlib inline
import os
import hydrofunctions as hf
import pandas as pd
import matplotlib.pyplot as plt
DEBUG = False

basin = 'merced'
# basin = 'upper san joaquin'
# basin = 'tuolumne'

variable = 'streamflow'
pc = '00060'

# variable = 'storage'
# pc = '00054'

if not os.path.exists(basin):
    os.mkdir(basin)

gages = pd.read_excel('USGS gauges.xlsx', sheet_name='{} - {}'.format(basin, variable), header=None, squeeze=True)

dfs = []
if DEBUG:
    gages = gages[:3]
for i, gage in enumerate(gages):
    gage_number = gage.strip().split(' ')[1]
    print('retrieving {} ({}%)'.format(gage, int((i+1) / len(gages) * 100)))
    herring = hf.NWIS(gage_number, 'dv', '1975-10-01', '2018-09-30', parameterCd=pc)
    try:
        herring.get_data()
    except Exception as err:
        print('Error: ', gage_number, err)
        continue
    df = herring.df()
    s = df[df.columns[0]]
    s.name = gage
    dfs.append(s)
df = pd.concat(dfs, axis=1)
df.index.name = 'Date'
if DEBUG:
    print(df.head())
    df.plot()
    plt.show()
else:
    if variable == 'storage':
        df = df * 1233.5 / 1e6
        path = '../../data/{} River/gauges/storage_mcm.csv'.format(basin.title())
        df.to_csv(path)
    else:
        path = '../../data/{} River/gauges/streamflow_cfs.csv'.format(basin.title())
        df.to_csv(path)
print('done!')

retrieving USGS 11264500 MERCED R A HAPPY ISLES BRIDGE NR YOSEMITE CA (16%)
retrieving USGS 11266500 MERCED R A POHONO BRIDGE NR YOSEMITE CA (33%)
retrieving USGS 11270900 MERCED R BL MERCED FALLS DAM NR SNELL CA (50%)
retrieving USGS 11271290 MERCED R A SHAFFER BRIDGE NR CRESSEY CA (66%)
retrieving USGS 11272500 MERCED R NR STEVINSON CA (83%)
retrieving USGS 11269700 EXCHEQUER PH A EXCHEQUER CA (100%)
done!
